In [1]:
path = []

In [2]:
import os, sys

sys.path.append("C:\\Users\\MG\\Desktop\\brain_diagnosis\\2-EDA")
from eda_helper import see_dir_size, EDA_helper

data = EDA_helper('stage_pre_train')

FileNotFoundError: [Errno 2] File b'../1-Dataset/dcm_label.csv' does not exist: b'../1-Dataset/dcm_label.csv'

In [6]:
windowed_imgs = data.get_window_imgs()

In [7]:
import cv2 as cv

save_dir = './1-test/'
if not(os.path.exists(save_dir)):
    os.mkdir(save_dir)
    
for i in range(len(windowed_imgs)):
    cv.imwrite(save_dir+str(data.list[i])+'.png', windowed_imgs[i])

In [9]:
!dir 1-test

 C 드라이브의 볼륨: Windows
 볼륨 일련 번호: CC3B-DB98

 C:\Users\MG\Desktop\brain_diagnosis\3-Preprocessing\1-test 디렉터리

2020-03-18  오후 07:15    <DIR>          .
2020-03-18  오후 07:15    <DIR>          ..
2020-03-18  오후 07:15            41,932 ID_07c9bea25.dcm.png
2020-03-18  오후 07:15            25,680 ID_6c5d82413.dcm.png
2020-03-18  오후 07:15            52,766 ID_731ac5f03.dcm.png
2020-03-18  오후 07:15            31,512 ID_b35e2d521.dcm.png
2020-03-18  오후 07:15            60,869 ID_e0ff5bec6.dcm.png
               5개 파일             212,759 바이트
               2개 디렉터리  438,607,884,288 바이트 남음


In [10]:
import glob
import numpy as np
file_dir = './1-test/'
save_dir = './2-test/'

def resize_and_norm_img(file_dir, save_dir, img_size=256):

    if not(os.path.exists(save_dir)):
        os.mkdir(save_dir)

    file_list = [f for f in glob.glob(file_dir+'/*.png', recursive=True)]
    for file in file_list:
        img = cv.imread(file, cv.IMREAD_GRAYSCALE)
        img = cv.normalize(img.astype(np.uint8), None, 0, 255, cv.NORM_MINMAX)
        img = cv.resize(img, dsize=(img_size, img_size), interpolation=cv.INTER_LINEAR)
        img_name = os.path.basename(file)
        img_name = os.path.splitext(img_name)[0]
        cv.imwrite(save_dir+img_name+'.png', img)

In [11]:
resize_and_norm_img(file_dir, save_dir)

In [13]:
!dir 2-test

 C 드라이브의 볼륨: Windows
 볼륨 일련 번호: CC3B-DB98

 C:\Users\MG\Desktop\brain_diagnosis\3-Preprocessing\2-test 디렉터리

2020-03-18  오후 07:16    <DIR>          .
2020-03-18  오후 07:16    <DIR>          ..
2020-03-18  오후 07:16            17,310 ID_07c9bea25.dcm.png
2020-03-18  오후 07:16            11,027 ID_6c5d82413.dcm.png
2020-03-18  오후 07:16            21,576 ID_731ac5f03.dcm.png
2020-03-18  오후 07:16            14,445 ID_b35e2d521.dcm.png
2020-03-18  오후 07:16            22,643 ID_e0ff5bec6.dcm.png
               5개 파일              87,001 바이트
               2개 디렉터리  438,607,949,824 바이트 남음


In [14]:
data.label_type

AttributeError: 'EDA_helper' object has no attribute 'label_type'

In [102]:
import matplotlib.pylab as plt
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

data_dir = './2-test/'


file_list = [f for f in glob.glob(file_dir+'/*.png', recursive=True)]
idx_list = [file_list[i].split('.')[1].split('/')[2] for i in range(len(file_list))]

x_list = []
for file in file_list:
    img = image.load_img(file, target_size = (256, 256))
    img = img_to_array(img)
    img = img / 255
    x_list.append(img)
    
x_train = np.array(x_list)

In [116]:
import pandas as pd

#load the label data
label_path = '../1-Dataset'
train = pd.read_csv(label_path + '/stage_pre_train.csv', sep=",", dtype = 'unicode')
train['Sub_type'] = train['ID'].str.split("_", n = 3, expand = True)[2]
train['PatientID'] = train['ID'].str.split("_", n = 3, expand = True)[1]

filter_id = train['PatientID'] == '6c5d82413'
filter_type = train['Sub_type'] == 'subdural'

y_train = train[filter_type]
y_train = np.array(y_train['Label'])

In [115]:
print(x_train.shape, y_train.shape)
print(y_train)

(5, 256, 256, 3) (5,)
['0' '1' '0' '0' '0']
